In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import root
from scipy.stats import norm

from math import log, sqrt, exp
from scipy import stats  
import numpy as np
import math

## Binary Tree

In [2]:
import numpy as np
import matplotlib.pyplot as plt

def american_option_pricing_binary_tree(S, K, r, sigma, T, N, is_call):
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)
    
    # initial binary tree
    V = np.zeros((N + 1, N + 1))
    V[:, -1] = np.maximum(is_call * S * np.power(u, N) - K, 0)  # boundary condition

    for j in range(N - 1, -1, -1):
        for i in range(j, -1, -1):
            V[i, j] = np.exp(-r * dt) * (p * V[i, j + 1] + (1 - p) * V[i - 1, j + 1])
            if is_call:
                V[i, j] = np.maximum(V[i, j], S * np.power(u, i) * np.power(d, j - i) - K)
            else:
                V[i, j] = np.maximum(V[i, j], K - S * np.power(u, i) * np.power(d, j - i))
    
    return V[0, 0]

# initial value
S = 100  
K = 90  
r = 0.02  
sigma = 0.2  
T = 10  
b = 0.01  
N = 50

call_price = american_option_pricing_binary_tree(S, K, r, sigma, T, N, True)
put_price = american_option_pricing_binary_tree(S, K, r, sigma, T, N, False)

print(f"American Call Option Price: {call_price:.2f}")
print(f"American Put Option Price: {put_price:.2f}")

American Call Option Price: 10.00
American Put Option Price: 1.67


## BAW model for call option

In [3]:
import numpy as np
from scipy.stats import norm

def black_scholes(S, K, T, r, sigma, option_type='call'):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == 'call':
        option_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        option_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    
    return option_price

def baw_option(S, K, T, r, sigma, b, option_type='call'):
    # Bjerksund-Stensland model parameters
    M = 2 * r / sigma ** 2
    N = 2 * b / sigma ** 2
    K_m = 1 - np.exp(-r * T)
    
    q2 = (-(N - 1) + np.sqrt((N - 1) ** 2 + 4 * M / K_m)) / 2
    S_star = K / (1 - 1 / q2)

    d1 = (np.log(S / S_star) + (b + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    A2 = (S_star * (1 - np.exp((b - r) * T) * norm.cdf(d1))) / q2

    if option_type == 'call':
        if S < S_star:
            option_price = black_scholes(S, K, T, r, sigma, option_type='call') + A2 * (S / S_star) ** q2
        else:
            option_price = S - K  
    elif option_type == 'put':
        if S < S_star:
            option_price = black_scholes(S, K, T, r, sigma, option_type='put') + A2 * (S / S_star) ** q2
        else:
            option_price = K - S  
    
    return option_price

# initial value
S = 100  
K = 90   
r = 0.02  
sigma = 0.3 
T = 1    
b = 0.02  

# Test both Call and Put options
call_price = baw_option(S, K, T, r, sigma, b, option_type='call')
put_price = baw_option(S, K, T, r, sigma, b, option_type='put')

print(f"American Call Option Price: {call_price:.2f}")
print(f"American Put Option Price: {put_price:.2f}")

American Call Option Price: 25.15
American Put Option Price: 13.36
